In [64]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

# Pkg.add("CoordinateTransformations")
# Pkg.add("RigidBodyDynamics")
# Pkg.add("MeshCatMechanisms")
# Pkg.add("MeshCat")

using CoordinateTransformations
using RigidBodyDynamics
using MeshCatMechanisms
using MeshCat
using Printf
using LinearAlgebra

  Activating project at `~/cmu/courses/16745_OCRL/zoe2_optimal_controller/julia`


In [65]:
vis = Visualizer()
# open(vis)  # open the visualizer in a separate tab/window
render(vis) # render the visualizer here inside the jupyter notebook


┌ Info: Listening on: 127.0.0.1:8729, thread id: 1
└ @ HTTP.Servers /home/hayden/.julia/packages/HTTP/4AUPl/src/Servers.jl:382
┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8729
└ @ MeshCat /home/hayden/.julia/packages/MeshCat/9QrxD/src/visualizer.jl:43


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), Set{HTTP.WebSockets.WebSocket}(), ip"127.0.0.1", 8729, HTTP.Servers.Server{HTTP.Servers.Listener{Nothing, Sockets.TCPServer}}(HTTP.Servers.Listener{Nothing, Sockets.TCPServer}(Sockets.InetAddr{Sockets.IPv4}(ip"127.0.0.1", 8729), "127.0.0.1", "8729", nothing, Sockets.TCPServer(RawFD(116) active)), nothing, Set{HTTP.Connections.Connection}(), Task (runnable) @0x00007740ae8d3d00, ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (0, 131121414830928, 131121462765168))), MeshCat.AnimationContext[]))

In [66]:
urdf_path = joinpath(@__DIR__, "zoe2.urdf")
robot = parse_urdf(urdf_path)
mvis = MechanismVisualizer(robot, URDFVisuals(urdf_path), vis)

MechanismVisualizer{MechanismState{Float64, Float64, Float64, TypeSortedCollections.TypeSortedCollection{Tuple{Vector{Joint{Float64, Revolute{Float64}}}}, 1}}, Visualizer}(MechanismState{Float64, Float64, Float64, …}(…), MeshCat Visualizer with path /meshcat at http://127.0.0.1:8729, 12)

In [76]:
# Create a state for the mechanism
state = MechanismState(robot)
q = configuration(state)

println("Initial configuration vector:")
println(q)
println("Number of degrees of freedom: ", length(q))

# Iterate over the joints to print their names
for (i, joint) in enumerate(joints(robot))
    println("Joint ", i, ": ", joint.name)
end

# Example: update a joint value
q[2] = 0.3
q[3] = -0.3

# Update the visualizer with the new configuration
set_configuration!(mvis, q)

# state = MechanismState(robot, randn(2), randn(2))
# t, q, v = simulate(state, 5.0);

Initial configuration vector:
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Number of degrees of freedom: 7
Joint 1: axle_roll_back_joint
Joint 2: axle_yaw_front_joint
Joint 3: axle_yaw_back_joint
Joint 4: wheel_front_right_joint
Joint 5: wheel_front_left_joint
Joint 6: wheel_back_right_joint
Joint 7: wheel_back_left_joint


In [95]:
include(joinpath(@__DIR__, "zoe2.jl"))

# Model parameters
model = (
    L = 1.91,  # robot length
    B = 1.64,  # robot width
    r = 0.325  # wheel radius
)

# Simulation parameters
params = (
    dt = 0.1, # seconds
    total_steps = 100,
    model = model
)

# Define our states
x = 0.0
y = 0.0
ψ = 0.0
θ_f = -0.4   # front steering angle (radians)
θ_r = -0.4   # rear steering angle (radians)

# Each state vector: [x_b, y_b, ψ, θ_f, θ_r]
Xsim = [zeros(5) for _ in 1:params.total_steps]
Xsim[1] = [x, y, ψ, θ_f, θ_r]

# Define our control parameters (start as straight motion)
ω_fl = 1.0
ω_fr = 1.0
ω_rl = 1.0
ω_rr = 1.0

# Each control vector: [ω_fl, ω_fr, ω_rl, ω_rr]
Usim = [zeros(4) for _ in 1:params.total_steps]
Usim[1] = [ω_fl, ω_fr, ω_rl, ω_rr]

# Compute the changes in the states based on the control and accumulate over steps
for i in 1:params.total_steps-1

    # println("Xsim[", i, "]: ", Xsim[i])
    # println("Usim[", i, "]: ", Usim[i])

    Xdot = zoe2_dynamics(params.model, Xsim[i], Usim[i], debug=false)
    # println("Xdot: ", Xdot)

    # Unpack the dynamics vector
    dx_b = Xdot[1]
    dy_b = Xdot[2]
    dψ = Xdot[3]
    dθ_f = Xdot[4]
    dθ_r = Xdot[5]

    # Update the state based on the dynamics
    x += dx_b * params.dt
    y += dy_b * params.dt
    ψ += dψ * params.dt
    θ_f += dθ_f * params.dt
    θ_r += dθ_r * params.dt

    # Set the next vector
    Xsim[i+1] = [x, y, ψ, θ_f, θ_r]
    Usim[i+1] = [ω_fl, ω_fr, ω_rl, ω_rr]
end

println("Simulation data:")
println("x_b, y_b, ψ, θ_f, θ_r")
for i in 1:params.total_steps
    @printf("%3d: %8.2f, %8.2f, %8.2f, %8.2f, %8.2f\n", i, Xsim[i][1], Xsim[i][2], Xsim[i][3], Xsim[i][4], Xsim[i][5])
end

animate_zoe2(Xsim, params.dt)


Simulation data:
x_b, y_b, ψ, θ_f, θ_r
  1:     0.00,     0.00,     0.00,    -0.40,    -0.40
  2:     0.00,     0.03,    -0.01,    -0.40,    -0.40
  3:     0.00,     0.07,    -0.03,    -0.40,    -0.40
  4:     0.00,     0.10,    -0.04,    -0.40,    -0.40
  5:     0.00,     0.13,    -0.06,    -0.40,    -0.40
  6:     0.01,     0.17,    -0.07,    -0.40,    -0.40
  7:     0.01,     0.20,    -0.09,    -0.40,    -0.40
  8:     0.01,     0.24,    -0.10,    -0.40,    -0.40
  9:     0.01,     0.27,    -0.12,    -0.40,    -0.40
 10:     0.02,     0.30,    -0.13,    -0.40,    -0.40
 11:     0.02,     0.34,    -0.15,    -0.40,    -0.40
 12:     0.03,     0.37,    -0.16,    -0.40,    -0.40
 13:     0.03,     0.40,    -0.18,    -0.40,    -0.40
 14:     0.04,     0.44,    -0.19,    -0.40,    -0.40
 15:     0.05,     0.47,    -0.21,    -0.40,    -0.40
 16:     0.05,     0.50,    -0.22,    -0.40,    -0.40
 17:     0.06,     0.53,    -0.24,    -0.40,    -0.40
 18:     0.07,     0.57,    -0.25,    -0.40

┌ Info: Listening on: 127.0.0.1:8753, thread id: 1
└ @ HTTP.Servers /home/hayden/.julia/packages/HTTP/4AUPl/src/Servers.jl:382
┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8753
└ @ MeshCat /home/hayden/.julia/packages/MeshCat/9QrxD/src/visualizer.jl:43


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), UInt8[0x83, 0xa7, 0x6f, 0x70, 0x74, 0x69, 0x6f, 0x6e, 0x73, 0x82  …  0x5f, 0x61, 0x6e, 0x69, 0x6d, 0x61, 0x74, 0x69, 0x6f, 0x6e], Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("robot" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("world" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("front_axle" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x66, 0x72, 0x6f, 0x6e, 0x74, 0x5f, 0x61, 0x78, 0x6c, 0x65], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("wheel_front_left" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x66, 0x72, 0x6f, 0x6e, 0x74, 0x5f, 0x6c, 0x65, 0x66, 0x74], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_wheel_front_left_joint" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6c, 0x65, 0x66, 0x74, 0x5f, 0x6a, 0x6f, 0x69, 0x6e, 0x74], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_4" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x34], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x34], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))))), "wheel_front_right" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x72, 0x6f, 0x6e, 0x74, 0x5f, 0x72, 0x69, 0x67, 0x68, 0x74], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_wheel_front_right_joint" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x69, 0x67, 0x68, 0x74, 0x5f, 0x6a, 0x6f, 0x69, 0x6e, 0x74], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_3" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x33], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x33], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))))), "after_axle_yaw_front_joint" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x72, 0x6f, 0x6e, 0x74, 0x5f, 0x6a, 0x6f, 0x69, 0x6e, 0x74], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_5" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x35], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x35], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))))), "after_chassis_joint" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x73, 0x73, 0x69, 0x73, 0x5f, 0x6a, 0x6f, 0x69, 0x6e, 0x74], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_1" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x6